In [39]:
working_directory = '/users/ironstein/documents/projects working directory/handwriting_recognition_using_neural_networks_on_FPGA_final_year project/image processing/ipython notebooks/python C extension testing/mark3'
working_directory = '/users/ironstein/Documents/projects_working_directory/handwriting_recognition/image_processing/ipython notebooks/python C extension testing/mark3'
import os
import time
import graphlab as gl

os.chdir(working_directory)

In [40]:
%%file test.c

#include <stdio.h>

void for_loop_test_without_register_values(int n) {
    int i;
    int j;
    for(i=0;i<n;i++);
}

void for_loop_test_with_register_values(int n) {
    
    int var = n;
    int i;
    int p;     
    
    for(i=0;i<var;i++){
        for(p=0;p<100000000;p++);
    }
}


Overwriting test.c


###generating shared object for the above C file (test.so)


In [41]:
os.system('cd ' + working_directory)
!clang -c test.c
!clang -o test.so -shared test.c
!file test.so

test.so: Mach-O 64-bit dynamically linked shared library x86_64


###python file to access the functions in test.so shared library

In [42]:
%%file _test.py

import numpy
import ctypes

_test_ = numpy.ctypeslib.load_library('test','.')

#specifying the required argument types for each function
_test_.for_loop_test_without_register_values.argtypes = [ctypes.c_int]
_test_.for_loop_test_without_register_values.restype = ctypes.c_void_p
_test_.for_loop_test_with_register_values.argtypes = [ctypes.c_int]
_test_.for_loop_test_with_register_values.restype = ctypes.c_void_p

def C_for_test_without_register_values(n) : 
    _test_.for_loop_test_without_register_values(n)
    
def C_for_test_with_register_values(n) : 
    _test_.for_loop_test_with_register_values(n)

Overwriting _test.py


In [43]:
def Python_for_test(n) : 
    for i in range(n) : 
        pass

###decorator for calculating time of execution of a function

In [44]:
def time_this_function(function) : 
    try : 
        time
    except : 
        import time
    
    def wrapper_function(*args,**kwargs) : 
        start_time = time.time()
        function(*args,**kwargs)
        elapsed_time = (time.time() - start_time)*1000
#         print(elapsed_time,end=' ms\n')
        return(elapsed_time)
        
    return wrapper_function

###testing Python and C functions for comparing the performance

In [ ]:
import _test

@time_this_function
def C_test_without_register_values(n) : 
    _test.C_for_test_without_register_values(n)

@time_this_function
def C_test_with_register_values(n) : 
    _test.C_for_test_with_register_values
    
@time_this_function
def Python_test(n) : 
    Python_for_test(n)
    
# a = C_test_with_register_values(1000)
# a

###time to compare the difference

In [ ]:
C_time_array_without_register_values = []
C_time_array_with_register_values = []
Python_time_array = []
count_array = []
i = 1

while(i <= 100000000) : 
    print(i)
    C_time_array_without_register_values.append(C_test_without_register_values(i))
    C_time_array_with_register_values.append(C_test_with_register_values(i))
    Python_time_array.append(Python_test(i))
    count_array.append(i)
    i *= 10

1
10
100
1000
10000
100000
1000000
10000000
100000000

###generate a CSV (comma seperated values) file for the recorded data, to be used in Graphlab Create

In [ ]:
def generate_csv(name='default.csv',**kwargs) : 
    csv_file = open(name,'w')
    
    keys = kwargs.keys()
    values = kwargs.values()
    
    for key in keys[:-1] : 
        csv_file.write(key),
        csv_file.write(','),
    csv_file.write(keys[-1])
    csv_file.write('\n')

    for i in range(len(values[0])) : 
        for j in range(len(values)-1) : 
            csv_file.write(str(values[j][i])),
            csv_file.write(','),
        csv_file.write(str(values[len(values)-1][i]))
        csv_file.write('\n')
        
    csv_file.close()

In [ ]:
# generate_csv(name='test.csv',c_time_without_register = C_time_array_without_register_values,\
#              c_time_with_register_values = C_time_array_with_register_values,\
#              python_time = Python_time_array,count = count_array)
generate_csv(name='test.csv',c_time = C_time_array_without_register_values,\
             python_time = Python_time_array,count = count_array)
dataset = gl.SFrame('test.csv')
dataset

In [ ]:
gl.canvas.set_target('ipynb')
dataset.show(view='Line Chart',y='python_time',x='count')